In [25]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

import numpy as np
import requests as rq
import io, h5py

In [26]:
data = rq.get('https://www.dropbox.com/s/c3umbo5y13sqcfp/synthetic_dataset.h5?raw=true')
data.raise_for_status()

with h5py.File(io.BytesIO(data.content), 'r') as dataset:
    x_train = np.array(dataset['X_train']).astype(np.float32).transpose([0, 2, 1])
    y_train = np.array(dataset['Y_train']).astype(np.float32)
    x_valid = np.array(dataset['X_valid']).astype(np.float32).transpose([0, 2, 1])
    y_valid = np.array(dataset['Y_valid']).astype(np.int32)
    x_test = np.array(dataset['X_test']).astype(np.float32).transpose([0, 2, 1])
    y_test = np.array(dataset['Y_test']).astype(np.int32)

print(x_train.shape, y_train.shape)

(21000, 200, 4) (21000, 12)


In [58]:
input_layer = layers.Input(shape=(200, 4))

conv1 = layers.Conv1D(filters=32, kernel_size=13, padding='same', use_bias='false')(input_layer)
BN = layers.BatchNormalization()(conv1)
act = layers.Activation('relu')(BN)
pool1 = layers.MaxPool1D(pool_size=6)(act)
drop1 = layers.Dropout(0.1)(pool1)

positions = tf.range(drop1.shape[1])
context = layers.Embedding(input_dim=drop1.shape[1], output_dim=drop1.shape[2])(positions)

contextual_meaning = tf.add(drop1, context)

query = layers.Dense(100, name='query')(contextual_meaning)
value = layers.Dense(100)(contextual_meaning)

attention, weights = layers.MultiHeadAttention(num_heads=8, key_dim=32, dropout=0.1)(query, value, return_attention_scores=True)

flat = layers.Flatten()(attention)
fcl = layers.Dense(512, activation='relu')(flat)
drop2 = layers.Dropout(0.5)(fcl)

output = layers.Dense(12, activation='sigmoid')(drop2)

In [65]:
def generate_model(d1, d2, d3, lr):
  input_layer = layers.Input(shape=(200, 4))

  conv1 = layers.Conv1D(filters=32, kernel_size=13, padding='same', use_bias='false')(input_layer)
  BN = layers.BatchNormalization()(conv1)
  act = layers.Activation('relu')(BN)
  pool1 = layers.MaxPool1D(pool_size=6)(act)
  drop1 = layers.Dropout(d1)(pool1)

  positions = tf.range(drop1.shape[1])
  context = layers.Embedding(input_dim=drop1.shape[1], output_dim=drop1.shape[2])(positions)

  contextual_meaning = tf.add(drop1, context)

  query = layers.Dense(100, name='query')(contextual_meaning)
  value = layers.Dense(100)(contextual_meaning)

  attention, weights = layers.MultiHeadAttention(num_heads=8, key_dim=32, dropout=d2)(query, value, return_attention_scores=True)

  flat = layers.Flatten()(attention)
  fcl = layers.Dense(512, activation='relu')(flat)
  drop2 = layers.Dropout(d3)(fcl)

  output = layers.Dense(12, activation='sigmoid')(drop2)

  model = Model(inputs=input_layer, outputs=output)
  model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(curve='ROC', name='auroc'), tf.keras.metrics.AUC(curve='PR', name='aupr')])
  return model

In [66]:
d1 = [0.2, 0.5]
d2 = [0.2, 0.5]
d3 = [0.25, 0.5]
lr = [0.01, 0.001, 0.0001]

for i in d1:
  for j in d2:
    for k in d3:
      for l in lr:
        model = generate_model(i, j, k, l)
        name = f'model-{i}-{j}-{k}-{l}'
        tensorboard = tf.keras.callbacks.TensorBoard(log_dir=f'logs\\{name}')
        checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=f'real_models\\{name}', monitor='val_loss', save_best_only=True)
        model.fit(x_train, y_train, epochs=40, validation_data=(x_valid, y_valid), callbacks=[tensorboard, checkpoint])


Epoch 1/40
657/657 [==============================] - 8s 10ms/step - loss: 1.1373 - auroc: 0.4989 - aupr: 0.1454 - val_loss: 0.4140 - val_auroc: 0.5047 - val_aupr: 0.1475


INFO:tensorflow:Assets written to: real_models\model-0.2-0.2-0.25-0.01/assets


INFO:tensorflow:Assets written to: real_models\model-0.2-0.2-0.25-0.01/assets


Epoch 2/40
657/657 [==============================] - 6s 9ms/step - loss: 0.4190 - auroc: 0.5093 - aupr: 0.1492 - val_loss: 0.4129 - val_auroc: 0.4912 - val_aupr: 0.1412


INFO:tensorflow:Assets written to: real_models\model-0.2-0.2-0.25-0.01/assets


...

Epoch 5/40
657/657 [==============================] - 6s 9ms/step - loss: 0.4165 - auroc: 0.5009 - aupr: 0.1461 - val_loss: 0.4129 - val_auroc: 0.5019 - val_aupr: 0.1447
Epoch 6/40
657/657 [==============================] - 6s 9ms/step - loss: 0.4174 - auroc: 0.4978 - aupr: 0.1461 - val_loss: 0.4131 - val_auroc: 0.4933 - val_aupr: 0.1410
Epoch 7/40
657/657 [==============================] - 6s 9ms/step - loss: 0.4164 - auroc: 0.4989 - aupr: 0.1463 - val_loss: 0.4130 - val_auroc: 0.4919 - val_aupr: 0.1418
Epoch 8/40
657/657 [==============================] - 6s 9ms/step - loss: 0.4169 - auroc: 0.5038 - aupr: 0.1488 - val_loss: 0.4129 - val_auroc: 0.4957 - val_aupr: 0.1421
Epoch 9/40
657/657 [==============================] - 6s 9ms/step - loss: 0.4174 - auroc: 0.4994 - aupr: 0.1462 - val_loss: 0.4129 - val_auroc: 0.5026 - val_aupr: 0.1461
Epoch 10/40
657/657 [==============================] - 6s 9ms/step - loss: 0.4155 - auroc: 0.4988 - aupr: 0.1449 - val_loss: 0.4128 - val_auroc: 0.504

INFO:tensorflow:Assets written to: real_models\model-0.2-0.2-0.25-0.01/assets


INFO:tensorflow:Assets written to: real_models\model-0.2-0.2-0.25-0.01/assets


Epoch 12/40
657/657 [==============================] - 6s 9ms/step - loss: 0.4153 - auroc: 0.4968 - aupr: 0.1446 - val_loss: 0.4129 - val_auroc: 0.4973 - val_aupr: 0.1440
Epoch 13/40
657/657 [==============================] - 6s 9ms/step - loss: 0.4174 - auroc: 0.4987 - aupr: 0.1471 - val_loss: 0.4127 - val_auroc: 0.5038 - val_aupr: 0.1463
Epoch 14/40
657/657 [==============================] - 6s 9ms/step - loss: 0.4156 - auroc: 0.5029 - aupr: 0.1470 - val_loss: 0.4130 - val_auroc: 0.4908 - val_aupr: 0.1416
Epoch 15/40
656/657 [============================>.] - ETA: 0s - loss: 0.4142 - auroc: 0.4991 - aupr: 0.1446

KeyboardInterrupt: ignored